In [ ]:
# Test script for classification
# Import packages

import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from os.path import exists
import numpy as np
import pandas as pd
import os
import sys
import importlib

# Good idea to add this folder to the json.settings file as "python.analysis.extraPaths".
sys.path.append('../functionScripts/')

import plotFunctions # totalCountsPlot, data_heatmap, correlation_plot
import analysisFunctions
from initFunctions import createDirs, debugReport, loadLightSheetData
import classifyFunctions
import helperFunctions

# Set Paths to data and output
dirDict = dict()
rootDir = 'C:\OneDrive\MEng\KwanLab\Lightsheet_cFos_Pipeline\\'
dirDict['atlasDir'] = rootDir + 'Atlas\\'
dirDict['dataDir'] = rootDir + 'Data\\'
dirDict['B1'] =       dirDict['dataDir'] + 'lightSheetV1\\'
dirDict['B2'] =       dirDict['dataDir'] + 'lightSheetV2Rev\\'   #3/6/23 - Looking at the new, Realigned batch 2 data. #Realigned
dirDict['B2_Orig'] =  dirDict['dataDir'] + 'lightSheetV2\\'
dirDict['B3'] =       dirDict['dataDir'] + 'lightSheetV3\\'      #3/6/23 - Batch 3 with MDMA

batchSplit = False          # Splits drugs from the first batch of data, from the second, from the 3rd. Batch 1 is labeled with 'a' (aSAL, aKET, aPSI), Batch 3 (cKET, MDMA)
splitTag  = ['a', '', 'c']  # Appended the to beginning of data from the first batch (PSI, KET, SAL -> aPSI, KET, aSAL).
testSplit = False           # Splits an individual drug for the sake of examining self-similarity
oldBatch2 = False

debugOutputs = False        # Saves csvs at intervals
scalingFactor = True        # Applies 1/total_cells as a scaling factor per mouse.
debug_ROI = ['Dorsal nucleus raphe']
outputFormat = 'png'

switchDir = dict(testSplit=testSplit, batchSplit=batchSplit, splitTag=splitTag, oldBatch2=oldBatch2, debugOutputs=debugOutputs, scalingFactor=scalingFactor, debug_ROI=debug_ROI, outputFormat=outputFormat)

# Make directories, and add their strings to the directory dictionary.
dirDict = createDirs(rootDir, switchDir, dirDict)

In [ ]:
# Reload in case anything updated in these functions
importlib.reload(classifyFunctions)
importlib.reload(plotFunctions)
importlib.reload(helperFunctions)

# Identify parameters of feature clustering, per
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html

classifyDict = dict()

# Parameters for pivoting the data
classifyDict['data'] = 'cell_density' #cell_density, count
classifyDict['feature'] = 'abbreviation'
classifyDict['label'] = 'drug'

# Parameters for feature scaling and aggregation
classifyDict['featurefilt'] = False
classifyDict['featureAgg'] = False
classifyDict['featureSel_scale'] = False # True, False
classifyDict['featureSel_linkage'] = 'average'  # 'average', 'complete', 'single', 'ward' (if euclidean)
classifyDict['featureSel_distance'] = 'correlation' # 'correlation, 'cosine', 'euclidean'
classifyDict['cluster_count'] = 100 # Number of clusters to generate. Not used at the moment.
classifyDict['cluster_thres'] = 0.3 # Anything closer than this is merged into a cluster

# Parameters for classification
classifyDict['model_featureScale'] = True # True, False
classifyDict['multiclass'] = 'multinomial' # 'ovr', 'multinomial'

classifyDict['model_featureSel'] = 'Univar' # 'Univar', 'mutInfo', 'RFE', 'None'
classifyDict['model_featureSel_k'] = [70, 75, 80]
# classifyDict['model_featureSel_k'] = [1, 2, 3]

# classifyDict['model'] = 'LogRegElastic'
classifyDict['model'] = 'LogRegL2'
# classifyDict['model'] = 'LogRegL1'

# ParamGrid Features - in instances where gridCV is set to true, these are the parameters that will be tested.
paramGrid = dict()
# paramGrid['classif__l1_ratio'] = [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1]          # used for ElasticNet
paramGrid['classif__C'] = [1, 10, 100]       # used for LogisticRegression
classifyDict['pGrid'] = paramGrid

classifyDict['shuffle'] = False
classifyDict['gridCV'] = True
classifyDict['CV_count'] = 8 # Number of folds for cross-validation
classifyDict['max_iter'] = 200

# Load Pickle
lightsheet_data = pd.read_pickle('lightsheet_data.pkl')

# Big Data - the dataset prior to filtering based on summary structures
# lightsheet_data = pd.read_pickle('lightsheet_data_big.pkl')
# lightsheet_data.rename(columns={'Brain Area': 'Brain_Area'}, inplace=True)

# Pure visualizations
# plotFunctions.data_heatmap(lightsheet_data_filt, classifyDict, dirDict)
# plotFunctions.data_heatmap_single(lightsheet_data, classifyDict, dirDict)

# Statistics
# plotFunctions.correlation_plot(lightsheet_data, classifyDict, dirDict)
# plotFunctions.distance_matrix(lightsheet_data, classifyDict, dirDict)
# plotFunctions.correlation_plot_hier(lightsheet_data, classifyDict, dirDict)

classifyFunctions.classifySamples(lightsheet_data, classifyDict, dirDict)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

dimRedTech = 1
# Generate example data
# data, labels = make_blobs(n_samples=64, centers=4, n_features=297, random_state=42)

if classifyDict['featureSel_filter']:
    pandasdf = helperFunctions.filter_and_agg_Data(lightsheet_data, classifyDict)

# if classifyDict['featureSel_agg']:
if 0:
    ls_data_agg = helperFunctions.agg_cluster(pandasdf, classifyDict, dirDict)
else:
    ls_data_agg = pandasdf.pivot(index='dataset', columns=classifyDict['feature'], values=classifyDict['data'])    

ls_data_agg = ls_data_agg.sample(frac=1)

# X, y, featureNames, numYDict = hf.reformatData(pandasdf, classifyDict)
X = np.array(ls_data_agg.values)
# y = np.array([x[0:-1] for x in np.array(ls_data_agg.index)])
yStr = np.array([x[0:-1] for x in np.array(ls_data_agg.index)])
yDict = dict(zip(np.unique(yStr), range(1, len(np.unique(yStr))+1)))
y = np.array([yDict[x] for x in yStr])

data = X
labels = y
dimCount = 3

# Perform PCA
if dimRedTech == 1:
    pca = PCA(n_components=dimCount)
    data_pca = pca.fit_transform(data)
    axesLabel = 'PC '
    plotTitle = "PCA Scatter Plot"
    dimRedTitle = f"Top{dimCount}_PCA"
elif dimRedTech == 2:
    lda = LinearDiscriminantAnalysis(n_components=dimCount)
    data_pca = lda.fit_transform(data, labels)
    axesLabel = 'LD '
    plotTitle = "LDA Scatter Plot"
    dimRedTitle = f"Top{dimCount}_LDA"


# Create a scatter plot of the first two PCs, color-coded by labels
PCAset = [[1, 2], [2, 3], [1, 3]]

fig, axes = plt.subplots(nrows=1, ncols=dimCount,
                    figsize=(18, 5))

for idx, PCset in enumerate(PCAset):

    unique_labels = np.unique(labels)
    colors = plt.cm.get_cmap('Dark2', len(unique_labels))

    for i, label in enumerate(unique_labels):
        axes[idx].scatter(data_pca[labels == label, PCset[0]-1], data_pca[labels == label, PCset[1]-1], color=colors(i), label=f"Label {label}")

    # Add title, legend, and axis labels
    axes[idx].set_title(plotTitle)
    axes[idx].set_xlabel(f"{axesLabel}{PCset[0]}")
    axes[idx].set_ylabel(f"{axesLabel}{PCset[1]}")
    axes[idx].legend(np.unique(yStr))

    plt.savefig(dirDict['classifyDir'] + dimRedTitle + '.png', dpi=300, format='png', bbox_inches='tight')

# Display the plot
plt.show()
